## Loading data

In [3]:
#loading needed library
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
from scipy.io import loadmat
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

#Loading the MNIST data set
mnist = loadmat("mnist-original.mat")
mnist_data = mnist["data"].T
mnist_label = mnist["label"][0].astype(int)
#mnist_label
#mnist_data

#normalize data
def normalize(data):
    mean = np.mean(data, axis=1, keepdims=True)
    std = np.std(data, axis=1, keepdims=True)
    data_normalized = (data - mean)/std
    return data_normalized
mnist_data_normalized = normalize(mnist_data)

#split the class
known_class = [0,1,2,3,4,5]
unknown_class = [6,7,8,9]
mnist_data_known = []
mnist_label_known = []
mnist_data_unknown = []
mnist_label_unknown = []
for i in range(len(mnist_data_normalized)):
    if(mnist_label[i] in known_class):
        mnist_data_known.append(mnist_data_normalized[i])
        mnist_label_known.append(mnist_label[i])
    else:
        mnist_data_unknown.append(mnist_data_normalized[i])
        mnist_label_unknown.append(mnist_label[i])
mnist_data_known = np.array(mnist_data_known)
mnist_data_unknown = np.array(mnist_data_unknown)
#mnist_data_known 
mnist_label_known = np.array(mnist_label_known)
mnist_label_unknown = np.array(mnist_label_unknown)
#mnist_label_unknown

#spliting into training and testing data
X_train, X_test, Y_train, Y_test = train_test_split(mnist_data_known,mnist_label_known, test_size=0.20, random_state=42)
X_test_with_unknown = np.vstack((X_test,mnist_data_unknown))
Y_train = Y_train.reshape(Y_train.shape[0],1)
Y_test_with_unknown = np.hstack((Y_test,mnist_label_unknown))
Y_test = Y_test.reshape(Y_test.shape[0],1)
Y_test_with_unknown = Y_test_with_unknown.reshape(Y_test_with_unknown.shape[0],1)
print("The shape of the training set feature matrix is:", X_train.shape)
print("The shape of the training label vector is:", Y_train.shape)
print("The shape of the test set feature matrix is:", X_test.shape)
print("The shape of the test label vector is:", Y_test.shape)
print("The shape of the test set with unknown class is:",X_test_with_unknown.shape)
print("The shape of the test label vector with unknown is:",Y_test_with_unknown.shape)

The shape of the training set feature matrix is: (33638, 784)
The shape of the training label vector is: (33638, 1)
The shape of the test set feature matrix is: (8410, 784)
The shape of the test label vector is: (8410, 1)
The shape of the test set with unknown class is: (36362, 784)
The shape of the test label vector with unknown is: (36362, 1)
[4]


## NN function

In [4]:
def distance(p1,p2):
     return np.sqrt(np.sum((p1-p2)**2))
#d = distance(X_train[1],X_test[0])
#print(d)
#print(X_train[1])
#print(X_test[0])

def predict(x_train,y_train, test_point,T):
    dist = {0:1e9,1:1e9,2:1e9,3:1e9,4:1e9,5:1e9}
    for i in range(len(x_train)):
        if(dist[y_train[i][0]]>distance(test_point,x_train[i])):
            dist[y_train[i][0]] = distance(test_point,x_train[i])
    after_sorting=sorted(dist.items(),key=lambda item:item[1])
    first_nearest = after_sorting[0][1]
    second_nearest = after_sorting[1][1]
    label_first = after_sorting[0][0]
    r = first_nearest / second_nearest
    if(r < T ):
        return label_first
    else:
        return -1

def accuracy(x_test,y_test,x_train,y_train,T):
    correct = 0.0
    for i in range(len(x_test)):
        pred = predict(x_train,y_train,x_test[i],T)
        if(y_test[i] in unknown_class):
            if(pred == -1):
                correct += 1
        else:
            if(pred == y_test[i]):
                correct += 1
    acc = correct / len(y_test)
    return acc
acc = accuracy(X_test,Y_test,X_train,Y_train,0.8)
print("The acc without unknown is:",acc)
acc = accuracy(X_test_with_unknown,Y_test_with_unknown,X_train,Y_train,0.8)
print("The acc with unknown is:",acc)
    

The acc without unknown is: 0.8562425683709869
The acc with unknown is: 0.8999504977723998
